In [43]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [51]:
np.random.seed(500)

Corpus = pd.read_csv(r"corpus.csv",encoding='latin-1')

Corpus['text'].dropna(inplace=True)

Corpus['text'] = [entry.lower() for entry in Corpus['text']]

Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

In [52]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [53]:
for index,entry in enumerate(Corpus['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

In [54]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [55]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [56]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'stun': 4279, 'even': 1525, 'sound': 4131, 'track': 4559, 'beautiful': 385, 'paint': 3167, 'mind': 2838, 'well': 4865, 'would': 4952, 'recomend': 3605, 'people': 3236, 'hate': 2054, 'video': 4768, 'game': 1870, 'music': 2930, 'play': 3310, 'cross': 1024, 'ever': 1529, 'best': 427, 'back': 328, 'away': 318, 'crude': 1027, 'take': 4374, 'fresh': 1830, 'step': 4219, 'guitar': 2004, 'orchestra': 3104, 'impress': 2246, 'anyone': 203, 'care': 645, 'listen': 2617, 'soundtrack': 4132, 'anything': 204, 'read': 3571, 'lot': 2661, 'review': 3722, 'say': 3842, 'figure': 1699, 'write': 4958, 'disagree': 1231, 'bit': 449, 'ultimate': 4636, 'masterpiece': 2753, 'timeless': 4506, 'year': 4977, 'beauty': 387, 'simply': 4019, 'refuse': 3629, 'price': 3416, 'tag': 4373, 'pretty': 3411, 'must': 2936, 'go': 1932, 'buy': 604, 'cd': 676, 'much': 2920, 'money': 2877, 'one': 3082, 'feel': 1675, 'worth': 4948, 'every': 1531, 'penny': 3235, 'amaze': 153, 'favorite': 1663, 'time': 4505, 'hand': 2024, 'intense': 

In [57]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  84.56666666666666
